In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Uploading dataset to the dataframe
df=pd.read_csv("daily_minimum_temps.csv",parse_dates=["Date"],index_col="Date")
df.head()

/tmp/ipython-input-2-2951785943.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df=pd.read_csv("daily_minimum_temps.csv",parse_dates=["Date"],index_col="Date")


,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


In [ ]:
#Finding out missing values
df.isna().sum(axis=0)

,0
Temp,0


In [ ]:
#Checking if any value in the dataset is string format
df["Temp"]=pd.to_numeric(df["Temp"],errors="coerce")#Coerce make unknown values to NaN
df.isna().sum(axis=0)

,0
Temp,3


In [5]:
df=df.dropna()
df.isna().sum(axis=0)

,0
Temp,0


In [6]:
#Normalize the Features
scaler=MinMaxScaler()
data_scaled=scaler.fit_transform(df["Temp"].values.reshape(-1,1))

In [7]:
#Sequence length for temperature data
seq_length=30

In [8]:
#Create a function for creating sequences
def create_sequences(data_scaled,seq_length):
    X,y=[],[]
    for i in range(len(data_scaled)-seq_length):
        X.append(data_scaled[i:i+seq_length])
        y.append(data_scaled[i+seq_length])
    return np.array(X),np.array(y)

In [9]:
#Calling the function and storing the values in X and y
X,y=create_sequences(data_scaled,seq_length)

In [10]:
#Divide the dataset into train test and split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=False)

In [11]:
#Building model
model=Sequential([
    LSTM(64,activation="relu",input_shape=(seq_length,1)),
    Dense(1)#Output layer. Since output is a single value
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.compile(optimizer="adam",loss="mse")

In [13]:
model.fit(X_train,y_train,epochs=20,batch_size=32)

Epoch 1/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 0.0927
Epoch 2/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0115
Epoch 3/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0112
Epoch 4/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0104
Epoch 5/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0109
Epoch 6/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0107
Epoch 7/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0103
Epoch 8/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0103
Epoch 9/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0102
Epoch 10/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0096
Epoch 11/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0103
Epoch 12/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0094
Epoch 13/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0099
Epoch 14/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0095
Epoch 15/20
91/91 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0093
Epoch 16/20
91/91 

In [14]:
#Make predictions
y_pred_scaled=model.predict(X_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [15]:
#Inverse Transform the scaled data
y_pred_scaled=np.clip(y_pred_scaled,0,1)
y_pred=scaler.inverse_transform(y_pred_scaled)
y_test_actual=scaler.inverse_transform(y_test)

In [16]:
#Predict the next day temperature
last_sequence = data_scaled[-seq_length:].reshape(1, seq_length, 1)
next_temp_scaled = model.predict(last_sequence)
next_temp_scaled=np.clip(next_temp_scaled,0,1)
next_day_temp = scaler.inverse_transform(next_temp_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


In [19]:
print("Predicted next day temperature is:",next_day_temp)

Predicted next day temperature is: [[13.411464]]
